# Siamese BiLSTM Neural Network with Attention

In [1]:
import sys
import torch
import logging
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from scipy.stats import pearsonr
from gensim.models import KeyedVectors
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

logging.disable(logging.WARNING)
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)

## Mono-Lingual Semantic Similarity

In [2]:
# Load all the data
train_path = './data/train.csv'
test_path = './data/test.csv'
val_path = './data/validation.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
val_data = pd.read_csv(val_path)

In [3]:
modelpath = "GoogleNews-vectors-negative300.bin"
model = KeyedVectors.load_word2vec_format(modelpath, binary=True)
word2idx = {word: i for i, word in enumerate(model.index_to_key)}

In [4]:
vocab = {}
j = 0
sentences_1 = train_data['sentence1'].apply(eval)
sentences_2 = train_data['sentence2'].apply(eval)

for i in range(len(sentences_1)):
    for word in sentences_1[i]:
        if word not in vocab and word in model.key_to_index:
            vocab[word] = j
            j += 1
    for word in sentences_2[i]:
        if word not in vocab and word in model.key_to_index:
            vocab[word] = j
            j += 1


word2idx_dataset = {}
for i in list(vocab.keys()):
    word2idx_dataset[vocab[i]] = word2idx[i]
word2idx_dataset['unk'] = len(word2idx_dataset)
word_indices = word2idx_dataset.values()
dataset_embed_matrix = model.vectors[np.array(list(word_indices))]

In [14]:
def pearson_corr(y_true, y_pred):
    corr, _ = pearsonr(y_true, y_pred)
    return corr

In [6]:
class CustomDataset(Dataset):
    def __init__(self, sentences1, sentences2, scores, word2idx):
        self.sentences1 = sentences1
        self.sentences2 = sentences2
        self.scores = scores
        self.word2idx = word2idx

    def __len__(self):
        return max(len(self.sentences1), len(self.sentences2))

    def __getitem__(self, idx):
        unk_token = self.word2idx['unk']
        sentence1 = self.sentences1[idx]
        sentence2 = self.sentences2[idx]
        score = self.scores[idx]
        seq1 = [self.word2idx[word] if word in self.word2idx else unk_token for word in sentence1]
        seq2 = [self.word2idx[word] if word in self.word2idx else unk_token for word in sentence2]
        return seq1, seq2, score

    def collate_fn(self, batch):
        sequences1, sequences2, scores = zip(*batch)
        padded_seqs1 = pad_sequence([torch.LongTensor(seq) for seq in sequences1], batch_first=True, padding_value=0)
        padded_seqs2 = pad_sequence([torch.LongTensor(seq) for seq in sequences2], batch_first=True, padding_value=0)
        return padded_seqs1, padded_seqs2, torch.LongTensor(scores)

In [7]:
train_data['sentence1'] = train_data['sentence1'].apply(eval)
train_data['sentence2'] = train_data['sentence2'].apply(eval)
val_data['sentence1'] = val_data['sentence1'].apply(eval)
val_data['sentence2'] = val_data['sentence2'].apply(eval)
test_data['sentence1'] = test_data['sentence1'].apply(eval)
test_data['sentence2'] = test_data['sentence2'].apply(eval)

In [8]:
batch_size = 16

train_dataset = CustomDataset(train_data['sentence1'], train_data['sentence2'], train_data['score'], word2idx_dataset)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_dataset.collate_fn)
val_dataset = CustomDataset(val_data['sentence1'], val_data['sentence2'], val_data['score'], word2idx_dataset)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=val_dataset.collate_fn)
test_dataset = CustomDataset(test_data['sentence1'], test_data['sentence2'], test_data['score'], word2idx_dataset)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=test_dataset.collate_fn)

In [9]:
class SiameseBiLSTM(nn.Module):
    def __init__(self, hidden_size, num_layers, embedding_dim, embd_matrix, dropout=0.2):
        super(SiameseBiLSTM, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding_dim = embedding_dim
        self.embd_matrix = embd_matrix

        self.word_embeddings = nn.Embedding(len(embd_matrix), embedding_dim)
        self.word_embeddings.weight = nn.Parameter(torch.from_numpy(self.embd_matrix))
        self.word_embeddings.weight.requires_grad = False

        self.bilstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.attention_fc = nn.Linear(hidden_size * 2, 1)
        self.attention_softmax = nn.Softmax(dim=1)
        self.fc = nn.Linear(hidden_size * 4, 1)  # 4 because we concatenate forward and backward hidden states of both LSTMs


    def forward_once(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.bilstm(embeds)
        lstm_out = self.dropout(lstm_out)
        attention_weights = self.attention_softmax(self.attention_fc(lstm_out))
        lstm_out = lstm_out * attention_weights
        lstm_out = lstm_out.sum(dim=1)
        return lstm_out
    

    def forward(self, sentence1, sentence2):
        output1 = self.forward_once(sentence1)
        output2 = self.forward_once(sentence2)
        concatenated = torch.cat((output1, output2), dim=1)
        similarity_score = torch.sigmoid(self.fc(concatenated))
        return similarity_score

In [10]:
# Define model and optimizer
model1 = SiameseBiLSTM(hidden_size=50, num_layers=2, embedding_dim=300, embd_matrix = dataset_embed_matrix)
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-3)
criterion = nn.MSELoss()
num_epochs = 10
model1.train()

for epoch in range(num_epochs):
    train_loss = 0
    for sentence1, sentence2, score in train_loader:
        sentence1_tensor = sentence1
        sentence2_tensor = sentence2
        score_tensor = torch.tensor(score, dtype=torch.float)/5
        optimizer.zero_grad()
        output = model1(sentence1_tensor, sentence2_tensor)
        loss = criterion(output.squeeze(), score_tensor.squeeze())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    print(f"Epoch = {epoch}\tTraining Loss = {train_loss/len(train_data)}")
    
    val_loss = 0
    with torch.no_grad():
        for sentence1, sentence2, score in val_loader:
            sentence1_tensor = sentence1
            sentence2_tensor = sentence2
            score_tensor = torch.tensor(score, dtype=torch.float)/5
            outputs = model1(sentence1_tensor, sentence2_tensor)
            val_loss = criterion(outputs.squeeze(), score_tensor.squeeze())
            val_loss += val_loss.item()
    
    print(f"Epoch = {epoch}\tValidation Loss = {val_loss/len(val_data)}")

Epoch = 0	Training Loss = 0.0051819710101453425
Epoch = 0	Validation Loss = 0.0001505515247117728
Epoch = 1	Training Loss = 0.005156482133938553
Epoch = 1	Validation Loss = 0.00011690163955790922
Epoch = 2	Training Loss = 0.005142274671175061
Epoch = 2	Validation Loss = 0.0001518282515462488
Epoch = 3	Training Loss = 0.005108761222886301
Epoch = 3	Validation Loss = 0.00014679056766908616
Epoch = 4	Training Loss = 0.0051102235885413134
Epoch = 4	Validation Loss = 0.0001506203698227182
Epoch = 5	Training Loss = 0.00509234941889305
Epoch = 5	Validation Loss = 0.00015986915968824178
Epoch = 6	Training Loss = 0.005103327176789757
Epoch = 6	Validation Loss = 0.000184585849638097
Epoch = 7	Training Loss = 0.005105185196915592
Epoch = 7	Validation Loss = 9.738941298564896e-05
Epoch = 8	Training Loss = 0.0051082442042376195
Epoch = 8	Validation Loss = 0.00015721259114798158
Epoch = 9	Training Loss = 0.005106234658949478
Epoch = 9	Validation Loss = 9.45208448683843e-05


In [11]:
train_predictions = []
train_labels = []
model1.eval()
for train_sentence1, train_sentence2, train_score in train_loader:
    train_sentence1_tensor = train_sentence1
    train_sentence2_tensor = train_sentence2
    train_score_tensor = torch.tensor(train_score, dtype=torch.float)/5.0
    train_output = model1(train_sentence1_tensor, train_sentence2_tensor)
    train_predictions.extend(train_output.tolist())
    train_labels.extend(train_score)
train_predictions = np.array(train_predictions)
train_labels = np.array(train_labels)
train_mse = mean_squared_error(train_labels, train_predictions)
print('Train MSE: {:.4f}'.format(train_mse))

Train MSE: 5.5902


In [12]:
val_predictions = []
val_labels = []
model1.eval()
for val_sentence1, val_sentence2, val_score in val_loader:
    val_sentence1_tensor = val_sentence1
    val_sentence2_tensor = val_sentence2
    val_score_tensor = torch.tensor(val_score, dtype=torch.float)/5.0
    val_output = model1(val_sentence1_tensor, val_sentence2_tensor)
    val_predictions.extend(val_output.tolist())
    val_labels.extend(val_score)
val_predictions = np.array(val_predictions)
val_labels = np.array(val_labels)
val_mse = mean_squared_error(val_labels, val_predictions)
print('Val MSE: {:.4f}'.format(val_mse))

Val MSE: 4.5013


In [13]:
test_predictions = []
test_labels = []
model1.eval()
for test_sentence1, test_sentence2, test_score in test_loader:
    test_sentence1_tensor = test_sentence1
    test_sentence2_tensor = test_sentence2
    test_score_tensor = torch.tensor(test_score, dtype=torch.float)/5.0
    test_output = model1(test_sentence1_tensor, test_sentence2_tensor)
    test_predictions.extend(test_output.tolist())
    test_labels.extend(test_score)
test_predictions = np.array(test_predictions)
test_labels = np.array(test_labels)
test_mse = mean_squared_error(test_labels, test_predictions)
print('Test MSE: {:.4f}'.format(test_mse))

Test MSE: 5.7274


In [14]:
corr = pearson_corr(train_labels, train_predictions.ravel())
corr

0.22443890007964687

In [15]:
corr = pearson_corr(val_labels, val_predictions.ravel())
corr

0.1192820668167186

In [16]:
corr = pearson_corr(test_labels, test_predictions.ravel())
corr

0.1243573168444356